# 실습 - 네이버 로그인 및 카페 크롤링

### 1. 네이버 로그인 계정 설정

In [1]:
# naverid = "네이버 로그인 아이디 입력"
# naverpw = "네이버 로그인 비밀번호 입력"

### 2.크롬 브라우저 설정

In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [3]:
dirver_loc = "d:/chromedriver/chromedriver.exe"

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
# 헤드리스 설정 시 리눅스 같이 웹화면 없는경우에도 웹페이지 실행가능함
#options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
# 웹 드라이버 정의
driver = webdriver.Chrome(dirver_loc, options=options)

### 3. 네이버 로그인

##### 1. 네이버 페이지 이동

In [4]:
import pyperclip

naver_page= 'https://www.naver.com/'

driver.get(naver_page)
time.sleep(1)

In [5]:
# 로그인 버튼을 찾고 클릭합니다
login_btn = driver.find_element_by_xpath('//*[@id="account"]/a')
login_btn.click()
time.sleep(1)

In [6]:
# id, pw 입력할 곳을 찾습니다.
tag_id = driver.find_element_by_xpath('//*[@id="id"]')
tag_pw = driver.find_element_by_xpath('//*[@id="pw"]')
tag_id.clear()
time.sleep(1)

# id 입력
tag_id.click()
# 클립보드 생성 복사
pyperclip.copy(naverid)
tag_id.send_keys(Keys.CONTROL, 'v')
time.sleep(2)

# pw 입력
tag_pw.click()
pyperclip.copy(naverpw)
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(2)

# 로그인 버튼을 클릭합니다
login_btn = driver.find_element_by_xpath('//*[@id="log.login"]')
login_btn.click()
time.sleep(2)

### 4. 카페 접속

In [7]:
keyword = "비대면"

#카페 접속 및 검색어 크롤링
cafe_url = 'https://cafe.naver.com/lovekopo'

driver.get(cafe_url)
time.sleep(3)

### 5. 검색어 타입 및 프레임 전환

In [8]:
from selenium.webdriver.common.keys import Keys

#driver.get_screenshot_as_file('naver_main_headless.png')

# 상단 우측 검색어입력 버튼 
search_input=driver.find_element_by_xpath('//*[@id="topLayerQueryInput"]')

#데이터 크롤링
search_input.send_keys(keyword)
search_input.send_keys(Keys.ENTER)

# 프레임 살펴보기
iframes = driver.find_elements_by_css_selector('iframe')
for iframe in iframes:
    print(iframe.get_attribute('name'))

driver.switch_to_default_content # 상위프레임으로 전환
driver.switch_to.frame("cafe_main") # 특정 프레임으로 전환



padding
padding
padding
padding

cafe_main
cafe_check

checkFrame



In [9]:
time.sleep(2)
html = driver.page_source

### 6. 페이지 크롤링

In [10]:
import bs4
import pandas as pd
bs = bs4.BeautifulSoup(html, "html.parser")

In [11]:
boardDiv = bs.find("div",attrs={"class":"article-board result-board m-tcol-c"})
boardTrs = boardDiv.find(name="table").find(name="tbody").findAll("tr")

In [12]:
rowList = []
columnList = []
for i in range(0, len(boardTrs)):
    if i%2!=0:
        continue
    rowNumber = boardTrs[i].find(name="div").find("div").text
    columnList.append(rowNumber)
    title = boardTrs[i].find(name="a").text.replace("\n","").replace("  ","")
    columnList.append(title)
    tds = boardTrs[i].findAll("td")
    for j in range(0, len(tds)):
        if (j==0) | (j==1):
            continue
        columnList.append(tds[j].text.replace("  "," ").replace("\n",""))
    rowList.append(columnList)
    columnList = []

pd.DataFrame(rowList)

,0,1,2,3,4
0,68704,[신소재응용] 대면수업 준비 완료,대구캠퍼스,2020.05.01.,220
1,68693,[컴퓨터응용기계과] 시제품제작실습 비대면 수업 진행 중,교직원 임대섭,2020.04.26.,87
2,68692,[컴퓨터응용기계과] 비대면 쌍방향 온라인 교육중,교직원 임대섭,2020.04.26.,91
3,68674,[신소재응용과]사회적 거리 두기에 적극 동참해 주시기 바랍니다.!!!,교직원 오세희,2020.04.20.,50
4,68661,"[그린건축과]""동행""을 통해서 배우는 인문학",교직원정동환,2020.04.17.,32
5,68649,"폴리텍대학 전문기술과정,1년과정, 국비과정 훈련수당 궁금하신분 보세요.",교직원 조명아,2020.04.10.,363
6,68646,[스마트전기과]비대면 수업 과제샘플,교원 김성삼,2020.04.09.,154
7,68645,[스마트전기과]학과 코로나 19 방역 활동,교원 김성삼,2020.04.09.,103
8,68610,[그린건축과] 비대면 수업연장과 관련해서.....,교직원정동환,2020.04.01.,231
9,68604,[컴퓨터응용기계과] 2020학년 기계과 신입생을 위한 학교생활 안내사항,교직원 임대섭,2020.03.31.,90
